<a href="https://colab.research.google.com/github/VivekKejriwal/TheSchoolOfAI/blob/main/Assignment_3ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#!pip install torch

In [23]:
import torch
import numpy as np
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.autograd
import torch.optim as optim
from torchsummary import summary

In [24]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1= nn.Conv2d(in_channels=1  ,out_channels=10 ,kernel_size=3)         #28X28X1  >>26x26X10    #RF 3
    self.conv2= nn.Conv2d(in_channels=10 ,out_channels=10 ,kernel_size=3)         #26X26X10 >>24X24X10    #RF 5 
    self.conv3= nn.Conv2d(in_channels=10 ,out_channels=20 ,kernel_size=3)         #12X12X10 >>10X10X20    #RF 12
    self.conv4= nn.Conv2d(in_channels=20 ,out_channels=20 ,kernel_size=3)         #10X10X20 >>8X8X20      #RF 14
    self.conv5= nn.Conv2d(in_channels=20 ,out_channels=30 ,kernel_size=3)         #4X4X20   >>2X2X30      #RF 30
    self.pool=  nn.AvgPool2d(2,2)                                                 #2X2X30   >>1X1X30      #RF 30
    self.out  = nn.Linear(in_features= 1*1*30,out_features=1000)                  #1X1X30   >>1X1000

  def forward(self,t):
    #input layer
    t=t
    
    #First hidden layer
    t=F.relu(self.conv1(t))
    t=F.relu(self.conv2(t))

    #1st maxpool layer
    t=F.max_pool2d(t, kernel_size=2, stride=2)

    #2nd hidden layer
    t=F.relu(self.conv3(t))
    t=F.relu(self.conv4(t))

    #2nd max-pool layer
    t=F.max_pool2d(t,kernel_size=2,stride=2)

    #3rd layer
    t=F.relu(self.conv5(t))

    #GAP layer
    t=self.pool(t)

    #Fully connected layer
    t=t.reshape(-1,1*1*30)
    t=self.out(t)

    return F.softmax(t,dim=1)


In [25]:
use_cuda = torch.cuda.is_available()   #USING GPU
device = torch.device("cuda" if use_cuda else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
network= Network().to(device)   #Transferring network to GPU

for name,param in network.named_parameters():
  print(name,'\t\t',param.shape)

conv1.weight 		 torch.Size([10, 1, 3, 3])
conv1.bias 		 torch.Size([10])
conv2.weight 		 torch.Size([10, 10, 3, 3])
conv2.bias 		 torch.Size([10])
conv3.weight 		 torch.Size([20, 10, 3, 3])
conv3.bias 		 torch.Size([20])
conv4.weight 		 torch.Size([20, 20, 3, 3])
conv4.bias 		 torch.Size([20])
conv5.weight 		 torch.Size([30, 20, 3, 3])
conv5.bias 		 torch.Size([30])
out.weight 		 torch.Size([1000, 30])
out.bias 		 torch.Size([1000])


In [26]:
#Defining the training set for EMNIST data

train_set = torchvision.datasets.EMNIST(
    root = './data',
    split = 'letters',
    train = True ,
    download = True,
    transform = transforms.Compose([
          transforms.ToTensor()
        ])
)

In [27]:
#Function to calculate number of correct predicitons in images

def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [28]:
#Defining the loader for training with batch size of 100

train_loader= torch.utils.data.DataLoader(train_set,batch_size=100, shuffle=True, **kwargs)

optimizer= optim.SGD(network.parameters(), lr=0.1, momentum=0.9) #Stochastic Gradient Descent 

for epoch in range(20):
  total_loss=0
  total_correct=0

  for batch in train_loader:  #Get Batch
    images, labels = batch
    images, labels= images.to(device), labels.to(device)
    preds= network(images) #Pass batch of images
    
    loss= F.cross_entropy(preds, labels) #Calculate Loss

    optimizer.zero_grad()
    loss.backward() #Calculate gradients
    optimizer.step() #Update the weights

    total_loss+= loss.item() #Add losses
    total_correct+= get_num_correct(preds, labels) #Add total number of correct predicitons
    
  print("epoch",epoch,"total correct:",total_correct,"loss:",total_loss)

epoch 0 total correct: 2137 loss: 8620.82691001892
epoch 1 total correct: 4760 loss: 8620.711944580078
epoch 2 total correct: 4851 loss: 8585.183608531952
epoch 3 total correct: 4800 loss: 8575.020342350006
epoch 4 total correct: 4800 loss: 8575.020339012146
epoch 5 total correct: 4800 loss: 8575.020339012146
epoch 6 total correct: 4800 loss: 8575.020346164703
epoch 7 total correct: 4800 loss: 8575.020344257355
epoch 8 total correct: 4800 loss: 8575.02035522461
epoch 9 total correct: 4800 loss: 8575.020342350006
epoch 10 total correct: 4800 loss: 8575.020337104797
epoch 11 total correct: 4800 loss: 8575.02035522461
epoch 12 total correct: 4800 loss: 8575.02033996582
epoch 13 total correct: 4800 loss: 8575.020347118378
epoch 14 total correct: 4800 loss: 8575.020342350006
epoch 15 total correct: 4800 loss: 8575.020347595215
epoch 16 total correct: 4800 loss: 8575.020338058472
epoch 17 total correct: 4800 loss: 8575.020342826843
epoch 18 total correct: 4800 loss: 8575.020337104797
epoch 1